In [ ]:
import numpy as np
from astropy.table import Table
from scipy.spatial import Delaunay
from itertools import combinations
import pandas as pd
import os

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.tri as mtri
import matplotlib.cm as cm
matplotlib.rcParams['figure.dpi'] = 360
matplotlib.rcParams['text.usetex'] = True
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']

### Data

In [ ]:
data_ngc = Table.read("create_files/QSO_NGC_clustering_data.ecsv", format="ascii.ecsv").to_pandas()
rand_ngc = Table.read("create_files/QSO_NGC_clustering_random.ecsv", format="ascii.ecsv").to_pandas()

data_sgc = Table.read("create_files/QSO_SGC_clustering_data.ecsv", format="ascii.ecsv").to_pandas()
rand_sgc = Table.read("create_files/QSO_SGC_clustering_random.ecsv", format="ascii.ecsv").to_pandas()

In [ ]:
data_ngc['RAN'] = False
rand_ngc['RAN'] = True

data_sgc['RAN'] = False
rand_sgc['RAN'] = True

In [ ]:
# Concatenate the data and random catalogs
df_north = pd.concat([data_ngc, rand_ngc], ignore_index=True)
df_south = pd.concat([data_sgc, rand_sgc], ignore_index=True)

In [ ]:
df_hemispheres = [df_north, df_south]

# High completeness regions

In [ ]:
max(df_hemispheres[1]['REDSHIFT'])

In [ ]:
df_NGC_1 = df_hemispheres[0][
    (df_hemispheres[0]['RA'] > 110) & (df_hemispheres[0]['RA'] < 260) &
    (df_hemispheres[0]['DEC'] > -10) & (df_hemispheres[0]['DEC'] < 8) 
    #(df_hemispheres[0]['REDSHIFT'] > 0.4) & (df_hemispheres[0]['REDSHIFT'] < 0.8)
]

df_NGC_2 = df_hemispheres[0][
    (df_hemispheres[0]['RA'] > 180) & (df_hemispheres[0]['RA'] < 260) &
    (df_hemispheres[0]['DEC'] > 30) & (df_hemispheres[0]['DEC'] < 40) 
    #(df_hemispheres[0]['REDSHIFT'] > 0.4) & (df_hemispheres[0]['REDSHIFT'] < 0.8)
]

df_SGC_3 = df_hemispheres[1]#[
    #(df_hemispheres[1]['REDSHIFT'] > 0.4) & (df_hemispheres[1]['REDSHIFT'] < 0.8)
#]

In [ ]:
df_subregions = [df_NGC_1, df_NGC_2, df_SGC_3]
labels_subregions = ['NGC-1', 'NGC-2', 'SGC-3']

### Delaunay triangulation

In [ ]:
tri_3d_list = []
tri_2d_list = []

for df in df_hemispheres:
    coords_3d = df[['X', 'Y', 'Z']].values
    coords_2d = df[['X', 'Y']].values

    tri_3d = Delaunay(coords_3d)
    tri_2d = mtri.Triangulation(df['X'], df['Y'])

    tri_3d_list.append(tri_3d)
    tri_2d_list.append(tri_2d)

In [ ]:
tri_3d_subregions = []
tri_2d_subregions = []

for df in df_subregions:
    coords_3d = df[['X', 'Y', 'Z']].values
    coords_2d = df[['X', 'Y']].values

    tri_3d = Delaunay(coords_3d)
    tri_2d = mtri.Triangulation(df['X'], df['Y'])

    tri_3d_subregions.append(tri_3d)
    tri_2d_subregions.append(tri_2d)

### For normal region

In [ ]:
x_min, x_max = -5800, -5400
y_min, y_max =  -80,    0

fig, axes = plt.subplots(1, 2, figsize=(18, 9))
axes = axes.flatten()

for i, (name, df, triang, ax) in enumerate(zip(['North', 'South'], df_hemispheres, tri_2d_list, axes)):

    #df_cut = df[(df['X'] > x_min) & (df['X'] < x_max) & 
     #           (df['Y'] > y_min) & (df['Y'] < y_max)]
    
    df_cut = df

    data_real = df_cut[~df_cut['RAN']]
    data_rand = df_cut[df_cut['RAN']]

    ax.triplot(triang, linewidth=0.5, color='black', alpha=0.3)

    ax.scatter(data_rand['X'], data_rand['Y'], s=0.1, c='blue', label='Random')
    ax.scatter(data_real['X'], data_real['Y'], s=0.3, c='red', label='Data')

    ax.set_title(f'{name} Region', fontsize=12)
    ax.set_xlabel('X [Mpc]')
    ax.set_ylabel('Y [Mpc]')
    #ax.set_xlim(x_min, x_max)
    #ax.set_ylim(y_min, y_max)
    ax.set_box_aspect(1)
    ax.legend(fontsize=8, loc='upper right')

plt.tight_layout()
plt.show()

### For filtered region

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

for label, df in zip(labels_subregions, df_subregions):
    if df.empty:
        print(f"[Omitido] {label} está vacío.")
        continue

    data_real = df[~df['RAN']]
    data_rand = df[df['RAN']]

    ax.scatter(data_rand['RA'], data_rand['DEC'], s=0.1, label=f'{label} Random', color='blue')
    ax.scatter(data_real['RA'], data_real['DEC'], s=0.01, alpha=0.6, label=f'{label} Data', color='red')

ax.set_title('All Subregions – RA vs DEC', fontsize=14)
ax.set_xlabel('RA [deg]')
ax.set_ylabel('DEC [deg]')
ax.set_xlim(0, 360)
ax.set_ylim(-90, 90)
ax.set_aspect('auto')
ax.legend(fontsize=8, loc='upper right', ncol=2)
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, len(tri_2d_subregions), figsize=(6 * len(tri_2d_subregions), 6))
if len(tri_2d_subregions) == 1:
    axes = [axes]  

for ax, label, df, triang in zip(axes, labels_subregions, df_subregions, tri_2d_subregions):

    data_real = df[~df['RAN']]
    data_rand = df[df['RAN']]

    ax.triplot(triang, linewidth=0.5, color='black', alpha=0.3)

    ax.scatter(data_rand['X'], data_rand['Y'], s=0.1, c='blue', label='Random')
    ax.scatter(data_real['X'], data_real['Y'], s=0.1, c='red', label='Data')

    ax.set_title(f'{label} Region', fontsize=12)
    ax.set_xlabel('X [Mpc]')
    ax.set_ylabel('Y [Mpc]')
    ax.set_box_aspect(1)
    ax.legend(fontsize=8, loc='upper right')

plt.tight_layout()
plt.show()

### Get $\space r$

In [ ]:
def compute_r(df):
    coords = df[['X', 'Y', 'Z']].values
    is_data = ~df['RAN'].values

    tri = Delaunay(coords)

    #! adjacency list for neighbors
    neighbors = {i: set() for i in range(len(coords))}
    for simplex in tri.simplices:
        for i, j in combinations(simplex, 2):
            neighbors[i].add(j)
            neighbors[j].add(i)

    r = np.zeros(len(coords), dtype=float)
    for i, nbrs in neighbors.items():
        n_data = int(np.sum(is_data[list(nbrs)]))
        n_rand = len(nbrs) - n_data
        if (n_data + n_rand) > 0:
            r[i] = (n_data - n_rand) / (n_data + n_rand)
        else:
            raise ValueError(f'No neighbors for point {i} in the triangulation.')

    out = df.copy()
    out['r'] = r
    return out

In [ ]:
df_r = [compute_r(df_hemispheres[0]), compute_r(df_hemispheres[1])] # [NORTH, SOUTH]

In [ ]:
df_r_subregions = [compute_r(df_subregions[0]), 
        compute_r(df_subregions[1]), 
        compute_r(df_subregions[2])] # ['NGC-1', 'NGC-2', 'SGC-3']

### CDF of $r$

In [ ]:
from statsmodels.distributions.empirical_distribution import ECDF

In [ ]:
colors = ['red', 'blue']
labels = ['North', 'South'] 

plt.figure(figsize=(10, 7))

for i in range(len(df_r)):
    r_data = df_r[i][~df_r[i]['RAN']]['r'].values
    r_rand = df_r[i][df_r[i]['RAN']]['r'].values

    cdf_data = ECDF(r_data)
    cdf_rand = ECDF(r_rand)

    r_data_sorted = cdf_data.x
    cdf_data_vals = cdf_data.y

    r_rand_sorted = cdf_rand.x
    cdf_rand_vals = cdf_rand.y

    color = colors[i]

    plt.plot(r_data_sorted, cdf_data_vals, color=color, linestyle='-', label=f'{labels[i]} Data')
    plt.plot(r_rand_sorted, cdf_rand_vals, color=color, linestyle='dotted', label=f'{labels[i]} Random')

plt.xlabel(r'$r\,[\mathrm{Mpc}]$')
plt.ylabel('CDF')
plt.title('QSO')
plt.grid(True)
plt.legend(ncol=2, fontsize='small', loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
colors = ['red', 'blue', 'green']

plt.figure(figsize=(10, 7))

for i in range(len(df_r_subregions)):
    r_data = df_r_subregions[i][~df_r_subregions[i]['RAN']]['r'].values
    r_rand = df_r_subregions[i][df_r_subregions[i]['RAN']]['r'].values

    cdf_data = ECDF(r_data)
    cdf_rand = ECDF(r_rand)

    r_data_sorted = cdf_data.x
    cdf_data_vals = cdf_data.y

    r_rand_sorted = cdf_rand.x
    cdf_rand_vals = cdf_rand.y

    color = colors[i]

    plt.plot(r_data_sorted, cdf_data_vals, color=color, linestyle='-', label=f'{labels_subregions[i]} Data')
    plt.plot(r_rand_sorted, cdf_rand_vals, color=color, linestyle='dotted', label=f'{labels_subregions[i]} Random')

plt.xlabel(r'$r\,[\mathrm{Mpc}]$')
plt.ylabel('CDF')
plt.title('QSO')
plt.grid(True)
plt.legend(ncol=2, fontsize='small', loc='upper left')
plt.tight_layout()
plt.show()

### Classify

In [ ]:
def classify_r(df):
    r = df['r'].values
    conds = [(r >= -1.0) & (r <= -0.9),
             (r > -0.9) & (r <= 0.0),
             (r > 0.0) & (r <= 0.9),
             (r > 0.9) & (r <= 1.0),]
    choices = ['void', 'sheet', 'filament', 'knot']
    df = df.copy()
    df['TYPE'] = np.select(conds, choices, default='error')
    return df

In [ ]:
df_typed = [classify_r(df_r[0]), classify_r(df_r[1])] # [NORTH, SOUTH]

In [ ]:
df_typed_subregion = [classify_r(df_r_subregions[0]), 
                      classify_r(df_r_subregions[1]), 
                      classify_r(df_r_subregions[2])] # ['NGC-1', 'NGC-2', 'SGC-3']

In [ ]:
structure_types = ['void', 'sheet', 'filament', 'knot']
hemisphere_labels = ['North', 'South'] 

dfs_by_hemisphere = []

for i, df in enumerate(df_typed):
    hemi_rows = []
    hemi_labels = []

    for source, label in [(False, f"{hemisphere_labels[i]} Data"), (True, f"{hemisphere_labels[i]} Random")]:
        df_sub = df[df['RAN'] == source]
        total = len(df_sub)
        values = []

        for t in structure_types:
            count = np.sum(df_sub['TYPE'] == t)
            if count == 0:
                frac = 0.0
                std = 0.0
            else:
                frac = count / total
                std = ((frac * (1 - frac)) / total) ** 0.5

            frac_percent = frac * 100
            std_percent = std * 100

            values.append(f"{frac_percent:.2f}% ± {std_percent:.2f}%")

        hemi_rows.append(values)
        hemi_labels.append(label)

    hemi_df = pd.DataFrame(hemi_rows, columns=['Voids', 'Sheets', 'Filaments', 'Knots'], index=hemi_labels)
    dfs_by_hemisphere.append(hemi_df)

for i, hemi_df in enumerate(dfs_by_hemisphere):
    print(f"\n{hemisphere_labels[i]}")
    display(hemi_df)


In [ ]:
dfs_by_subregions = []

for i, df in enumerate(df_typed_subregion):
    rows = []
    labels = []

    for source, label in [(False, f"{labels_subregions[i]} Data"), (True, f"{labels_subregions[i]} Random")]:
        df_sub = df[df['RAN'] == source]
        total = len(df_sub)
        values = []

        for t in structure_types:
            count = np.sum(df_sub['TYPE'] == t)
            if count == 0:
                frac = 0.0
                std = 0.0
            else:
                frac = count / total
                std = ((frac * (1 - frac)) / total) ** 0.5

            frac_percent = frac * 100
            std_percent = std * 100

            values.append(f"{frac_percent:.2f}% ± {std_percent:.2f}%")

        rows.append(values)
        labels.append(label)

    subregions_df = pd.DataFrame(rows, columns=['Voids', 'Sheets', 'Filaments', 'Knots'], index=labels)
    dfs_by_subregions.append(subregions_df)

for i, subregions_df in enumerate(dfs_by_subregions):
    print(f"\n{labels_subregions[i]}")
    display(subregions_df)


### Plot

In [ ]:
type_colors = {
    'void': 'black',
    'sheet': 'blue',
    'filament': 'red',
    'knot': 'green'
}

# For data

In [ ]:
xlims = [(-4500, -2500), (3500, 5500)]
ylims = [(-200,200), (-200,200)]

for h_idx, df in enumerate(df_typed):
    label = hemisphere_labels[h_idx]
    coords2d = df[['X', 'Y']].values
    is_real = ~df['RAN'].values

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))  
    axes = axes.flatten()

    for t_idx, tp in enumerate(structure_types):
        color = type_colors[tp]
        mask_t = (df['TYPE'] == tp).values
        mask_real = mask_t & is_real

        ax = axes[t_idx]

        ax.scatter(
            coords2d[mask_real, 0], coords2d[mask_real, 1],
            s=1, facecolors=color, edgecolors='black', linewidths=0.3, alpha=0.4
        )
        ax.set_title(tp.capitalize(), fontsize=12)
        ax.set_xlabel('X [Mpc]')
        ax.set_ylabel('Y [Mpc]')
        ax.set_xlim(xlims[h_idx])
        ax.set_ylim(ylims[h_idx])
        ax.set_box_aspect(1)

    plt.suptitle(f'QSO - {label} Hemisphere (Data)', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


# For random

In [ ]:
xlims = [(-4500, -2500), (3500, 5500)]
ylims = [(-200,200), (-200,200)]

for h_idx, df in enumerate(df_typed):
    label = hemisphere_labels[h_idx]
    coords2d = df[['X', 'Y']].values
    is_real = ~df['RAN'].values

    fig, axes = plt.subplots(1, 4, figsize=(20, 5))  
    axes = axes.flatten()

    for t_idx, tp in enumerate(structure_types):
        color = type_colors[tp]
        mask_t = (df['TYPE'] == tp).values
        mask_rand = mask_t & ~is_real

        ax = axes[t_idx]

        ax.scatter(
            coords2d[mask_rand, 0], coords2d[mask_rand, 1],
            s=1, facecolors=color, edgecolors='black', linewidths=0.3, alpha=0.4
        )
        ax.set_title(tp.capitalize(), fontsize=12)
        ax.set_xlabel('X [Mpc]')
        ax.set_ylabel('Y [Mpc]')
        ax.set_xlim(xlims[h_idx])
        ax.set_ylim(ylims[h_idx])
        ax.set_box_aspect(1)

    plt.suptitle(f'QSO - {label} Hemisphere (Random)', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()